<a href="https://colab.research.google.com/github/dimanyen/card-benefits/blob/main/%E4%BF%A1%E7%94%A8%E5%8D%A1%E5%84%AA%E6%83%A0GAI_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package

In [1]:
!pip install langchain chromadb openai langchain-community pypdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.7/987.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.7/328.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

# 建立向量資料庫

In [2]:
! unzip /content/信用卡優惠訊息-20240719T073550Z-001.zip

Archive:  /content/信用卡優惠訊息-20240719T073550Z-001.zip
  inflating: 信用卡優惠訊息/星展銀行.pdf  
  inflating: 信用卡優惠訊息/信用卡優惠懶人包(保費、燃料費、王品、首刷禮、機場貴賓室).pdf  
  inflating: 信用卡優惠訊息/匯豐銀行.pdf  
  inflating: 信用卡優惠訊息/中國信託.pdf  
  inflating: 信用卡優惠訊息/玉山銀行.pdf  
  inflating: 信用卡優惠訊息/台新銀行.pdf  
  inflating: 信用卡優惠訊息/國泰世華.pdf  


In [3]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from google.colab import drive
from langchain.docstore.document import Document
import chromadb
from chromadb.config import Settings
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

# 掛載Google Drive
# drive.mount('/content/drive', force_remount=True)

# 設置資料夾路徑
folder_path = "/content/信用卡優惠訊息"

# 初始化PDF文件加載器
pdf_loaders = []

# 遍歷資料夾內所有PDF文件
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        pdf_loaders.append(PyPDFLoader(file_path))

# 確認有無PDF文件
if not pdf_loaders:
    raise FileNotFoundError(f"No PDF files found in directory: {folder_path}")

# 加載所有PDF文件
documents = []
for loader in pdf_loaders:
    documents.extend(loader.load())

# 確認有無加載文檔
if not documents:
    raise ValueError("No documents were loaded.")

# 分割文本
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=30
)
texts = text_splitter.split_documents(documents)

# 確認有無分割文本
if not texts:
    raise ValueError("Text splitting failed.")

# 初始化OpenAI嵌入
embeddings = OpenAIEmbeddings(api_key=api_key)

# 提取文本內容
document_texts = [doc.page_content for doc in texts]

# 確認有無文本內容
if not document_texts:
    raise ValueError("No text content found in documents.")

# 嵌入文檔
document_embeddings = embeddings.embed_documents(document_texts)

# 確認有無嵌入文檔
if not document_embeddings:
    raise ValueError("Document embedding failed.")

# 初始化Chroma客戶端
chroma_client = chromadb.Client(Settings(persist_directory="/content/vector_store_new"))

# 創建Chroma集合
collection = chroma_client.create_collection(name="document_collection")

# 添加文本和對應的嵌入到Chroma集合
for i, (embedding, text) in enumerate(zip(document_embeddings, document_texts)):
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        metadatas=[{"text": text}]
    )

print("Documents have been successfully added to the Chroma vector store.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Documents have been successfully added to the Chroma vector store.


In [6]:
query = "買pchome, momo，有沒有什麼優惠"
# 將查詢轉換為嵌入向量
query_embedding = embeddings.embed_query(query)

# 在Chroma中搜索相似文檔
results = collection.query(query_embeddings=[query_embedding], n_results=5)

# 獲取相似文檔的文本
similar_texts = [metadata["text"] for metadata in results["metadatas"][0]]

display(similar_texts)

['國際電商： Coupang 酷澎(台灣)、淘寶 /天貓。 \n【樂饗購】指定消費 含 \n國內百貨：遠東 SOGO百貨 、太平洋百貨、新光三越、 SKM Park 、遠東百貨、\nBig City遠東巨城購物中心、微風廣場、誠品生活  、環球購物中心、 CITYLINK 、\nBELLAVITA 、統一時代、台北 101、ATT 4 FUN 、明曜百貨、京站、美麗華、大葉\n高島屋、比漾廣場、大江國際購物中心、中友百貨、廣三 SOGO、Tiger City 、勤',
 '##國泰世華 CUBE卡 \n#3.3%網購 /百貨 /餐旅等超過 100家通路，享最高 3.3%回饋無上限  \n☆ 持帳戶及 CUBE APP 享指定消費 3%回饋無上限  ☆ \n【玩數位】指定消費 含 \n數位/串流平台： Apple媒體服務  、Google Play 、Disney+、Netflix、Spotify、\nKKBO X \n網購：蝦皮購物、 momo購物網、 PChome 線上購物、博客來、小樹 購 \n國際電商： Coupang 酷澎(台灣)、淘寶 /天貓。',
 '一般消費滿百 HAPPY GO 點 \nHAPPY GO 特約商店點數 2 \n指定商店回饋點數 3倍',
 '*每人每 30天內限用乙次；部分商品不適用  \n*與第二件免費、第二件五折、其他現折優惠不可併用  \n*每筆訂單之折扣碼使用上限為  1 組，無法多組合併使用。  \n(2024/9/30 止) \n#$800新戶享小樹點 (信用卡 )800點 \n\uf0b7 商家：國內一般消費、海外一般消費',
 '最高 5%現金回饋 (需登錄，含原國外實體商店消費 2.5%現金回饋 )，每戶加碼上\n限300元/月，每月限量 5,000名 \n※國外實體商店消費定義：該筆交易請款時，消費地或商店登記之國別碼為國\n外(非台灣 )，且須為面對面交易 (包括以實體卡、 Apple Pay 、Google Pay 、\nSamsung Pay 於實體商店支付 )；網路交易、條碼支付或第三方支付不列入消費\n計算。相關交易判斷最終以中國信託 銀行及國際組織系統資料為準。  \n#指定商品 7折(2024/12/31 止)']

In [7]:
from openai import OpenAI
from IPython.display import Markdown, display

client = OpenAI(api_key=api_key)
completion = client.chat.completions.create(
  model="gpt-4o-mini-2024-07-18",
  messages=[
    {"role": "system", "content": "根據我輸入的問題與內容生成結果"},
    {"role": "user", "content": f'問題:{query}\n內容:{similar_texts}'}
  ]
)

display(Markdown(completion.choices[0].message.content))

在PChome和momo購物網站上，目前有幾個優惠可以利用：

1. **信用卡回饋**：使用國泰世華的CUBE卡進行網購，可享有3.3%的最高回饋，並且沒有上限。持有帳戶及CUBE APP的用戶在指定消費時，還可以獲得3%的回饋。

2. **HAPPY GO點數回饋**：一般消費滿百的時候，可以累積HAPPY GO點數，有些特約商店提供的回饋點數高達3倍。

3. **新戶優惠**：如果是首次購物，也可以享有最高$800的新戶優惠，並可以在特定信用卡消費的時候，獲得小樹點數。

4. **現金回饋方案**：還有其他現金回饋方案，最多可達5%，但是需要事先登錄，而且有每月的加碼限制。

5. **指定商品折扣**：某些指定商品會有7折優惠，截止日期到2024年12月31日。

建議在進行購物時，查看具體的促銷活動和條件，以獲得最佳優惠。

# 輸入UI

In [8]:
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00


In [32]:
# 詢問問題，並透過RAG回答
def ask_question(question):
  # 將查詢轉換為嵌入向量
  query_embedding = embeddings.embed_query(query)

  # 在Chroma中搜索相似文檔
  results = collection.query(query_embeddings=[query_embedding], n_results=5)

  # 獲取相似文檔的文本
  similar_texts = [metadata["text"] for metadata in results["metadatas"][0]]

  return similar_texts

def to_llm_prompt(query, similar_texts):
  client = OpenAI(api_key=api_key)
  completion = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    # model="gpt-4o",
    messages=[
      {"role": "system", "content": "根據我輸入的問題與內容產生關於信用卡優惠訊息"},
      {"role": "user", "content": f'問題:{query}\n內容:{similar_texts}'}
    ]
  )
  return completion.choices[0].message.content


In [33]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# 創建輸入框
input_box = widgets.Text(
    description='信用卡優惠:',
    placeholder='輸入你的問題',
)
# 創建按鈕
button = widgets.Button(
    description='詢問'
)

# 創建Output小部件來顯示loading
output = widgets.Output()

# 定義按鈕的事件處理函數
def on_button_click(b):
  with output:
    # 顯示loading指示器
    clear_output(wait=True)
    display("處理中，請稍候...")

    query = input_box.value
    similar_texts = ask_question(query)
    result = to_llm_prompt(query, similar_texts)
    display(Markdown(result))

# 綁定按鈕的點擊事件
button.on_click(on_button_click)

# 顯示輸入框和按鈕
box = widgets.HBox([input_box, button])
display(box, output)


Output()